In [ ]:
pip install requests beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Fungsi untuk mengambil link berita dari halaman Kompas Regional
def get_news_links(page_url):
    response = requests.get(page_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        # Mencari semua link berita di halaman
        news_links = soup.find_all("a", href=True)
        # Filter hanya link yang mengandung 'regional.kompas.com/read/'
        regional_news_links = [link['href'] for link in news_links if 'nasional.kompas.com/read/' in link['href']]
        return regional_news_links
    else:
        print(f"Gagal mengakses halaman. Status code: {response.status_code}")
        return []

# URL halaman utama Kompas Regional
base_url = "https://nasional.kompas.com/"
news_links = []
page_num = 1
max_pages = 10  # Maksimal halaman yang akan dibuka

# Loop untuk mengambil link berita sampai mendapatkan 200 link atau maksimal halaman tercapai
while len(news_links) < 1000 and page_num <= max_pages:
    # URL untuk halaman dengan pagination (asumsi URLnya berformat seperti ini)
    page_url = f"{base_url}?page={page_num}"
    print(f"Mengambil link dari halaman: {page_url}")

    # Mengambil link dari halaman saat ini
    new_links = get_news_links(page_url)

    if not new_links:
        # Jika tidak ada link baru yang ditemukan, hentikan pencarian
        break

    news_links.extend(new_links)

    # Menghilangkan duplikasi link
    news_links = list(set(news_links))

    # Meningkatkan nomor halaman untuk pagination
    page_num += 1

# Menyimpan link ke file CSV menggunakan pandas
df = pd.DataFrame(news_links, columns=["link"])
df.to_csv('kompas_nasional.csv', index=False)

print(f"Total {len(news_links)} link telah disimpan ke 'news_links.csv'")


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Fungsi untuk mengambil konten dari sebuah artikel
def crawl_article(url):
    url = url + '?page=all'  # menambahkan '?page=all' untuk mendapatkan seluruh konten tanpa paging
    req_berita = requests.get(url)
    soup_berita = BeautifulSoup(req_berita.text, 'lxml')

    # Mengambil judul
    try:
        title = soup_berita.find_all('title')[0].text
    except:
        title = 'Tidak ada judul'

    # Mengambil tanggal
    try:
        tgl = soup_berita.find_all('div', {'class': 'read__time'})[0].text
        date = tgl.split(',')[0].split('-')[1].replace(' ', "")
    except:
        date = 'Tidak ada tanggal'

    # Mengambil isi artikel
    try:
        berita = soup_berita.find_all('div', {'class': 'read__content'})[0].text.strip().replace('\n', ' ')
    except:
        berita = 'Tidak ada konten'

    return {
        'title': title,
        'date': date,
        'content': berita
    }

# Membaca file CSV yang berisi daftar link
#file_path = '/content/news_links_Makassar.csv'
#file_path = '/content/news_links_Medan.csv'
#file_path = '/content/news_links_Surabaya.csv'
#file_path = '/content/news_links_Yogyakarta.csv'
#file_path = '/content/news_links_bandung.csv'
#file_path = '/content/news_links_denpasar.csv'
file_path = '/content/kompas_nasional.csv'
df = pd.read_csv(file_path)

article_links = df['link'].tolist()

all_articles = []
for link in article_links:
    print(f"Memproses: {link}")
    article_content = crawl_article(link)
    all_articles.append(article_content)

    print(f"Title: {article_content['title']}")
    print(f"Date: {article_content['date']}")
    print(f"Content: {article_content['content'][:200]}...")
    print("="*80)

df_articles = pd.DataFrame(all_articles)

output_file = '/content/kompas_nasional.xlsx'
df_articles.to_excel(output_file, index=False)

print(f"Total {len(all_articles)} artikel telah disimpan ke '{output_file}'")